# CCC Bridge Residual Analysis

Comprehensive bridge analysis for CCC clock applications with ε-continuation sweeps,
commutator diagnostics, and validation of acceptance criterion A2.

This notebook provides:
- ε-continuation analysis with residual E(R) = ||∏exp{-ε(N_e-RD_e)} - I||
- Enhanced commutator diagnostics and α factor calculations
- Spectral vs Frobenius norm comparisons
- Exact-null panel demonstrations
- Weight optimization analysis

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

from bridge_ccc import CCCBridgeAnalyzer, CCCBridgeResult, epsilon_continuation
from metrology import CCCMetrology, PARAMETER_SETS

# Configure plotting
plt.style.use('default')
%matplotlib inline

print("🔬 CCC Bridge Residual Analysis")
print("=" * 40)

## 1. Bridge Edge Creation and Initial Analysis

In [ ]:
# Initialize CCC bridge analyzer
analyzer = CCCBridgeAnalyzer(use_spectral_norm=False)

# Create CCC-specific bridge edges
print("📊 Creating CCC bridge edges...")
n_dim = 4  # Matrix dimension
n_edges = 5  # Number of edges
complexity_factor = 2.0  # Complexity scaling

edges = analyzer.create_ccc_edges(
    n_dim=n_dim, 
    n_edges=n_edges, 
    complexity_factor=complexity_factor
)

print(f"✅ Created {len(edges)} edges of dimension {edges[0].N.shape[0]}")

# Initial commutator diagnostics
print("\n🔍 Initial commutator diagnostics...")
comm_diag = analyzer.enhanced_commutator_diagnostics(edges)

print(f"Commutator structure: {comm_diag.get('commutator_structure', 'unknown')}")
print(f"max_NN_comm = {comm_diag.get('max_NN_comm', 0):.3e}")
print(f"max_DD_comm = {comm_diag.get('max_DD_comm', 0):.3e}")
print(f"max_ND_comm = {comm_diag.get('max_ND_comm', 0):.3e}")
print(f"ND_comm_mean = {comm_diag.get('ND_comm_mean', 0):.3e}")
print(f"ND_comm_std = {comm_diag.get('ND_comm_std', 0):.3e}")

## 2. ε-Continuation Analysis

In [ ]:
# Perform comprehensive ε-continuation analysis
print("📈 Performing ε-continuation analysis...")

eps_analysis = analyzer.epsilon_continuation_analysis(
    edges,
    eps_start=1e-2,
    eps_target=1e-9,
    eps_factor=0.6
)

print(f"✅ Analysis complete:")
print(f"   R* = {eps_analysis['R_star']:.8f}")
print(f"   Final residual = {eps_analysis['residuals'][-1]:.3e}")
print(f"   Final α = {eps_analysis['final_alpha']:.2f}")
print(f"   Converged: {eps_analysis['converged']}")
print(f"   Steps: {len(eps_analysis['eps_values'])}")

In [ ]:
# Create comprehensive ε-sweep visualization
fig_eps = make_subplots(
    rows=2, cols=2,
    subplot_titles=('ε-Continuation Sweep', 'Residual vs ε (log-log)', 
                   'R* Convergence', 'α Factor Evolution'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

eps_vals = eps_analysis['eps_values']
residuals = eps_analysis['residuals']
R_vals = eps_analysis['R_values']
alpha_vals = eps_analysis['alpha_estimates']

# 1. ε-continuation sweep
fig_eps.add_trace(
    go.Scatter(x=eps_vals, y=residuals, mode='lines+markers',
               name='Residual', line=dict(width=3, color='blue')),
    row=1, col=1
)

# Add theoretical floor
max_ND_comm = comm_diag.get('max_ND_comm', 1e-12)
theoretical_floor = [eps * max_ND_comm for eps in eps_vals]
fig_eps.add_trace(
    go.Scatter(x=eps_vals, y=theoretical_floor, mode='lines',
               name='Theoretical Floor', line=dict(dash='dash', color='red')),
    row=1, col=1
)

fig_eps.update_xaxes(type="log", title_text="ε", row=1, col=1)
fig_eps.update_yaxes(type="log", title_text="Residual", row=1, col=1)

# 2. Log-log plot for linearity check
fig_eps.add_trace(
    go.Scatter(x=eps_vals, y=residuals, mode='lines+markers',
               name='Residual (log-log)', line=dict(width=3, color='green'),
               showlegend=False),
    row=1, col=2
)

# Fit line to check linearity
log_eps = np.log10(eps_vals)
log_residuals = np.log10(residuals)
slope, intercept = np.polyfit(log_eps[-5:], log_residuals[-5:], 1)  # Fit last 5 points
fit_line = 10**(slope * log_eps + intercept)

fig_eps.add_trace(
    go.Scatter(x=eps_vals, y=fit_line, mode='lines',
               name=f'Linear fit (slope={slope:.2f})', 
               line=dict(dash='dot', color='orange'), showlegend=False),
    row=1, col=2
)

fig_eps.update_xaxes(type="log", title_text="ε", row=1, col=2)
fig_eps.update_yaxes(type="log", title_text="Residual", row=1, col=2)

# 3. R* convergence
fig_eps.add_trace(
    go.Scatter(x=list(range(len(R_vals))), y=R_vals, mode='lines+markers',
               name='R* values', line=dict(width=3, color='purple'),
               showlegend=False),
    row=2, col=1
)

fig_eps.add_hline(y=eps_analysis['R_star'], line_dash="dash", line_color="red",
                  annotation_text=f"Final R* = {eps_analysis['R_star']:.6f}", 
                  row=2, col=1)

fig_eps.update_xaxes(title_text="ε-continuation step", row=2, col=1)
fig_eps.update_yaxes(title_text="R*", row=2, col=1)

# 4. α factor evolution
fig_eps.add_trace(
    go.Scatter(x=eps_vals, y=alpha_vals, mode='lines+markers',
               name='α estimates', line=dict(width=3, color='brown'),
               showlegend=False),
    row=2, col=2
)

fig_eps.add_hline(y=eps_analysis['final_alpha'], line_dash="dash", line_color="red",
                  annotation_text=f"Final α = {eps_analysis['final_alpha']:.2f}", 
                  row=2, col=2)

fig_eps.update_xaxes(type="log", title_text="ε", row=2, col=2)
fig_eps.update_yaxes(title_text="α factor", row=2, col=2)

fig_eps.update_layout(
    height=800,
    title_text="CCC Bridge ε-Continuation Analysis",
    showlegend=True
)

fig_eps.show()

print(f"\n📊 ε-sweep analysis:")
print(f"   Linear slope in log-log: {slope:.2f} (should be ~1 for linear floor)")
print(f"   R* convergence: {abs(R_vals[-1] - R_vals[-2]):.2e} (final change)")
print(f"   α factor stability: {np.std(alpha_vals[-3:]):.2f} (last 3 values)")

## 3. Spectral vs Frobenius Norm Comparison

In [ ]:
# Compare Frobenius and spectral norm analyses
print("🔍 Comparing Frobenius vs Spectral norm analysis...")

# Frobenius norm analysis (already done)
analyzer_fro = CCCBridgeAnalyzer(use_spectral_norm=False)
result_fro = analyzer_fro.analyze_ccc_bridge(edges)

# Spectral norm analysis
analyzer_spec = CCCBridgeAnalyzer(use_spectral_norm=True)
result_spec = analyzer_spec.analyze_ccc_bridge(edges)

print(f"\n📈 Comparison Results:")
print(f"{'Metric':<20} {'Frobenius':<15} {'Spectral':<15} {'Ratio':<10}")
print("-" * 65)
print(f"{'R*':<20} {result_fro.R_star:<15.8f} {result_spec.R_star:<15.8f} {result_spec.R_star/result_fro.R_star:<10.4f}")
print(f"{'Final residual':<20} {result_fro.residual:<15.3e} {result_spec.residual:<15.3e} {result_spec.residual/result_fro.residual:<10.4f}")
print(f"{'α factor':<20} {result_fro.alpha_factor:<15.2f} {result_spec.alpha_factor:<15.2f} {result_spec.alpha_factor/result_fro.alpha_factor:<10.4f}")
print(f"{'SE(R*)':<20} {result_fro.se_estimate:<15.3e} {result_spec.se_estimate:<15.3e} {result_spec.se_estimate/result_fro.se_estimate:<10.4f}")

In [ ]:
# Visualize norm comparison
from bridge_ccc import compute_residual_with_eps

# Generate residual landscapes for both norms
R_center = result_fro.R_star
R_range = 0.1 * abs(R_center)
R_grid = np.linspace(R_center - R_range, R_center + R_range, 200)
eps_test = result_fro.eps_final

residuals_fro = [compute_residual_with_eps(edges, R, eps_test) for R in R_grid]

# For spectral norm, we need to use the spectral residual function
# This is a simplified version - in practice would use the actual spectral norm function
residuals_spec = [compute_residual_with_eps(edges, R, eps_test) * 0.8 for R in R_grid]  # Approximate scaling

fig_norms = go.Figure()

fig_norms.add_trace(
    go.Scatter(x=R_grid, y=residuals_fro, mode='lines',
               name='Frobenius norm', line=dict(width=3, color='blue'))
)

fig_norms.add_trace(
    go.Scatter(x=R_grid, y=residuals_spec, mode='lines',
               name='Spectral norm (approx)', line=dict(width=3, color='red', dash='dash'))
)

fig_norms.add_vline(x=result_fro.R_star, line_dash="dot", line_color="blue",
                    annotation_text=f"R* (Fro) = {result_fro.R_star:.6f}")
fig_norms.add_vline(x=result_spec.R_star, line_dash="dot", line_color="red",
                    annotation_text=f"R* (Spec) = {result_spec.R_star:.6f}")

fig_norms.update_layout(
    title="Residual Landscape: Frobenius vs Spectral Norm",
    xaxis_title="R",
    yaxis_title="Residual",
    yaxis_type="log",
    height=500
)

fig_norms.show()

## 4. Exact-Null Panel Analysis

In [ ]:
# Create identical copies for exact-null demonstration
print("🔬 Exact-null panel analysis...")

# Import the function to create identical copies
try:
    from experiments_bridge_null import create_identical_copies
    
    # Create identical copies
    edges_identical = create_identical_copies(edges, len(edges))
    
    # Analyze identical copies
    result_identical = analyzer_fro.analyze_ccc_bridge(edges_identical)
    
    print(f"\n📊 Heterogeneous vs Identical Comparison:")
    print(f"{'Metric':<25} {'Heterogeneous':<15} {'Identical':<15} {'Ratio':<10}")
    print("-" * 70)
    print(f"{'R*':<25} {result_fro.R_star:<15.8f} {result_identical.R_star:<15.8f} {abs(result_identical.R_star-result_fro.R_star)/abs(result_fro.R_star):<10.4f}")
    print(f"{'Final residual':<25} {result_fro.residual:<15.3e} {result_identical.residual:<15.3e} {result_identical.residual/result_fro.residual:<10.4f}")
    print(f"{'α factor':<25} {result_fro.alpha_factor:<15.2f} {result_identical.alpha_factor:<15.2f} {result_identical.alpha_factor/result_fro.alpha_factor:<10.4f}")
    print(f"{'max_ND_comm':<25} {result_fro.commutator_norms.get('max_ND_comm', 0):<15.3e} {result_identical.commutator_norms.get('max_ND_comm', 0):<15.3e} {'N/A':<10}")
    
    # Check if identical copies achieve exact null
    exact_null_achieved = result_identical.residual < 1e-12
    print(f"\n✅ Exact null achieved: {'YES' if exact_null_achieved else 'NO'}")
    print(f"   Residual reduction: {result_fro.residual / result_identical.residual:.1f}x")
    
except ImportError:
    print("⚠️  Exact-null analysis not available (experiments_bridge_null not imported)")
    exact_null_achieved = False

## 5. Weight Optimization Analysis

In [ ]:
# Weight optimization to minimize residual floor
print("🎯 Weight optimization analysis...")

try:
    from experiments_bridge_null import optimize_weights
    
    # Optimize weights
    print("   Running weight optimization (this may take a moment)...")
    optimal_weights, optimal_residual = optimize_weights(
        edges, eps=result_fro.eps_final, max_iter=200, step_size=0.02, tol=1e-6
    )
    
    # Compare with uniform weights
    uniform_weights = np.ones(len(edges)) / len(edges)
    
    print(f"\n📈 Weight Optimization Results:")
    print(f"{'Metric':<20} {'Uniform':<15} {'Optimal':<15} {'Improvement':<12}")
    print("-" * 67)
    print(f"{'Residual':<20} {result_fro.residual:<15.6e} {optimal_residual:<15.6e} {result_fro.residual/optimal_residual:<12.2f}x")
    
    print(f"\n🎯 Optimal Weights:")
    for i, (w_uniform, w_optimal) in enumerate(zip(uniform_weights, optimal_weights)):
        print(f"   Edge {i+1}: {w_uniform:.4f} → {w_optimal:.4f} (change: {(w_optimal-w_uniform)/w_uniform*100:+.1f}%)")
    
    # Visualize weight optimization
    fig_weights = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Weight Comparison', 'Residual Improvement')
    )
    
    # Weight comparison
    edge_indices = list(range(1, len(edges) + 1))
    fig_weights.add_trace(
        go.Bar(x=edge_indices, y=uniform_weights, name='Uniform', 
               marker_color='lightblue', opacity=0.7),
        row=1, col=1
    )
    fig_weights.add_trace(
        go.Bar(x=edge_indices, y=optimal_weights, name='Optimal', 
               marker_color='darkblue', opacity=0.7),
        row=1, col=1
    )
    
    # Residual comparison
    methods = ['Uniform', 'Optimal']
    residuals_comp = [result_fro.residual, optimal_residual]
    fig_weights.add_trace(
        go.Bar(x=methods, y=residuals_comp, name='Residual', 
               marker_color=['lightcoral', 'darkred'], showlegend=False),
        row=1, col=2
    )
    
    fig_weights.update_xaxes(title_text="Edge Index", row=1, col=1)
    fig_weights.update_yaxes(title_text="Weight", row=1, col=1)
    fig_weights.update_xaxes(title_text="Method", row=1, col=2)
    fig_weights.update_yaxes(title_text="Residual", type="log", row=1, col=2)
    
    fig_weights.update_layout(
        height=400,
        title_text="Weight Optimization Analysis",
        showlegend=True
    )
    
    fig_weights.show()
    
except ImportError:
    print("⚠️  Weight optimization not available (experiments_bridge_null not imported)")
    optimal_residual = result_fro.residual

## 6. A2 Acceptance Criterion Validation

In [ ]:
# Validate A2 acceptance criterion
print("✅ A2 Acceptance Criterion Validation")
print("=" * 45)

# Validate bridge analysis results
validation = analyzer_fro.validate_bridge_analysis(result_fro)

print(f"\n📋 A2 Criterion Components:")
print(f"   R* reported: {'✅' if validation['R_star_reported'] else '❌'} (R* = {result_fro.R_star:.8f})")
print(f"   SE reported: {'✅' if validation['SE_reported'] else '❌'} (SE = {result_fro.se_estimate:.3e})")
print(f"   α reported: {'✅' if validation['alpha_reported'] else '❌'} (α = {result_fro.alpha_factor:.2f})")
print(f"   ε-sweep consistent: {'✅' if validation['eps_sweep_consistent'] else '❌'} (converged: {result_fro.converged})")
print(f"   Commutator analysis: {'✅' if validation['commutator_floor_analysis'] else '❌'}")

# Additional consistency checks
print(f"\n🔍 Additional Consistency Checks:")
linear_floor_check = abs(slope - 1.0) < 0.5  # ε-sweep should be approximately linear
alpha_reasonable = 0.1 < result_fro.alpha_factor < 10.0  # α should be reasonable
residual_floor_check = result_fro.residual > 1e-15  # Should hit commutator floor, not machine precision

print(f"   Linear ε-sweep: {'✅' if linear_floor_check else '❌'} (slope = {slope:.2f}, should be ~1)")
print(f"   Reasonable α: {'✅' if alpha_reasonable else '❌'} (α = {result_fro.alpha_factor:.2f})")
print(f"   Commutator floor: {'✅' if residual_floor_check else '❌'} (residual = {result_fro.residual:.3e})")

# Overall A2 assessment
a2_components = [
    validation['R_star_reported'],
    validation['SE_reported'], 
    validation['alpha_reported'],
    validation['eps_sweep_consistent'],
    validation['commutator_floor_analysis'],
    linear_floor_check,
    alpha_reasonable,
    residual_floor_check
]

a2_met = all(a2_components)
a2_score = sum(a2_components)

print(f"\n{'='*45}")
print(f"A2 CRITERION: {'✅ PASS' if a2_met else '❌ FAIL'} ({a2_score}/{len(a2_components)} checks passed)")

if not a2_met:
    print("\n❌ Issues found:")
    for issue in validation['issues']:
        print(f"   - {issue}")
else:
    print("\n✅ Bridge analysis reports R* with SE and α; ε-sweep consistent with linear floor")
    if 'exact_null_achieved' in locals() and exact_null_achieved:
        print("✅ Exact-null panel demonstrates zero residual floor")
    else:
        print("✅ Converged to commutator floor with diagnostics")

## 7. Summary and Export

In [ ]:
# Create comprehensive summary
print("📊 Bridge Analysis Summary")
print("=" * 30)

summary_data = {
    'Analysis_Type': ['Frobenius_Norm', 'Spectral_Norm'],
    'R_star': [result_fro.R_star, result_spec.R_star],
    'Final_Residual': [result_fro.residual, result_spec.residual],
    'Alpha_Factor': [result_fro.alpha_factor, result_spec.alpha_factor],
    'SE_Estimate': [result_fro.se_estimate, result_spec.se_estimate],
    'Converged': [result_fro.converged, result_spec.converged],
    'Final_Eps': [result_fro.eps_final, result_spec.eps_final]
}

df_summary = pd.DataFrame(summary_data)
print("\n📋 Analysis Results Summary:")
print(df_summary.to_string(index=False, float_format='%.6g'))

# Commutator diagnostics summary
comm_summary = {
    'Metric': ['max_NN_comm', 'max_DD_comm', 'max_ND_comm', 'ND_comm_mean', 'ND_comm_std'],
    'Value': [
        comm_diag.get('max_NN_comm', 0),
        comm_diag.get('max_DD_comm', 0),
        comm_diag.get('max_ND_comm', 0),
        comm_diag.get('ND_comm_mean', 0),
        comm_diag.get('ND_comm_std', 0)
    ]
}

df_comm = pd.DataFrame(comm_summary)
print("\n🔍 Commutator Diagnostics:")
print(df_comm.to_string(index=False, float_format='%.3e'))

# ε-continuation summary
eps_summary = {
    'eps_start': eps_analysis['eps_values'][0],
    'eps_final': eps_analysis['eps_values'][-1],
    'n_steps': len(eps_analysis['eps_values']),
    'residual_reduction': eps_analysis['residuals'][0] / eps_analysis['residuals'][-1],
    'R_star_stability': np.std(eps_analysis['R_values'][-3:]),
    'linear_slope': slope
}

print(f"\n📈 ε-Continuation Summary:")
for key, value in eps_summary.items():
    print(f"   {key}: {value:.3e}")

# Save results
df_summary.to_csv('../figures/bridge_analysis_results.csv', index=False)
df_comm.to_csv('../figures/commutator_diagnostics.csv', index=False)

# Save ε-continuation data
eps_data = pd.DataFrame({
    'eps': eps_analysis['eps_values'],
    'residual': eps_analysis['residuals'],
    'R_star': eps_analysis['R_values'],
    'alpha_estimate': eps_analysis['alpha_estimates']
})
eps_data.to_csv('../figures/eps_continuation_data.csv', index=False)

print("\n💾 Results saved to:")
print("   ../figures/bridge_analysis_results.csv")
print("   ../figures/commutator_diagnostics.csv")
print("   ../figures/eps_continuation_data.csv")

# Final A2 status
print(f"\n{'='*50}")
print(f"A2 ACCEPTANCE CRITERION: {'✅ SATISFIED' if a2_met else '❌ NOT SATISFIED'}")
print(f"Bridge analysis complete with comprehensive diagnostics.")
print(f"{'='*50}")